In [4]:
import FinanceDataReader as fdr
import yfinance as yf
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime, timedelta

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'GLOBAL'
ID = 'root'
PW = 'root'

nasdaq = list(fdr.StockListing('NASDAQ')['Symbol'])[:100]
sp = list(fdr.StockListing('SP500')['Symbol'])[:100]
etc = ['NRGU', 'FNGU', 'TQQQ']
symbols = list(set(nasdaq + sp + etc))

### 1. 나스닥, SP500, 기타종목 최신화
print(">> 1. Update Nasdq, sp500, etc")

### 2. 기존 데이터 제거
print("2. Delete old data")
db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
cursor.execute("TRUNCATE stocks_price")
db.commit()
db.close()

for idx in range(0, len(symbols), 50):
  symbol_list = symbols[idx:idx+50]
  df = yf.download(symbol_list, start = '2021-08-01')
  time.sleep(0.1)
  swapDf = df.swaplevel(0, 1, axis=1)
  for symbol in symbol_list:
    symbolDf = swapDf[symbol].rename(columns={'Open': 'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}).drop(['Adj Close'], axis=1)
    symbolDf['symbol'] = symbol
    symbolDf = symbolDf[['symbol', 'open', 'high', 'low', 'close', 'volume']]
    try:
      db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
      conn = db_connection.connect()
      symbolDf.to_sql(name='stocks_price', con=db_connection, if_exists='append', index=True, index_label="date")
      conn.close()
    except Exception as ex:
      print(ex, symbol)
      pass

# df = fdr.DataReader('BTC/KRW', '2021-07-01').rename(columns={'Open': 'open', 'High':'high', 'Low':'low', 'Close':'close', 'Volume':'volume'}).drop(['Change'], axis=1)
# df['symbol'] = 'BTC/KRW'
# df = df[['symbol', 'open', 'high', 'low', 'close', 'volume']]


100%|██████████| 4255/4255 [00:05<00:00, 780.52it/s]
C:\Users\Jack\AppData\Local\Programs\Python\Python38-32\lib\site-packages\FinanceDataReader\wikipedia\listing.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Symbol'] = df['Symbol'].str.replace('\.', '')


>> 1. 나스닥, SP500, 기타종목 최신화
2. 기존 데이터 제거
[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  50 of 50 completed

1 Failed download:
- BRKB: No data found, symbol may be delisted
[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  35 of 35 completed

1 Failed download:
- BFB: No data found, symbol may be delisted


In [ ]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime
import traceback

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'GLOBAL'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 2. 볼린저밴드 초기세팅
print(">> 2. Init bollinger band")

# 2.0. 기존 데이터 제거
print("2.0. Delete old data")
cursor.execute("TRUNCATE boll")
db.commit()

# 2.1. 심볼리스트 조회
print("2.1. Get symbol list")
cursor.execute("SELECT DISTINCT symbol FROM stocks_price")
symbols = list(map(lambda x: x[0], cursor.fetchall()))

# 2.2. 기간 설정
print("2.2. Set period")
start = '20210801'
end = str(datetime.now().date()).replace('-', '')
print("Period: (%s - %s)" %(start, end))

# 2.3. 새로운 데이터 업데이트
print("2.3. Update new data (%s - %s)" %(start, end))
day = 20
for symbol in symbols:
  try:
    cursor.execute("SELECT * FROM stocks_price WHERE symbol = '" + symbol + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'symbol', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'}).drop(['open', 'high', 'low', 'volume'], axis=1)
    # Set Dataframe
    priceDf['low'] = 0
    priceDf['medium'] = 0
    priceDf['high'] = 0
    priceDf['bandWidth'] = 0.0
    priceDf['position'] = 0.0
    # Set bandWidth, Bollinger
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      copyDf = priceDf[idx:idx+day].loc[:, 'close'].copy()
      avg = copyDf.mean()
      std = copyDf.std()
      
      lo = avg - std * 2
      me = avg
      hi = avg + std * 2
      bw = (hi - lo) / me
      pos = (priceDf.iloc[idx]['close'] - lo) / (hi - lo)
      priceDf.at[idx, 'low'] = lo
      priceDf.at[idx, 'medium'] = me
      priceDf.at[idx, 'high'] = hi
      priceDf.at[idx, 'bandWidth'] = bw
      priceDf.at[idx, 'position'] = pos

    bollDf = priceDf.reset_index().set_index('date').drop(['index'], axis=1)
    
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    bollDf.to_sql(name='boll', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', traceback.format_exc(), ex1)
    pass

In [14]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from pykrx import stock
import time
from datetime import datetime

ADDR = '192.168.56.100'
PORT = '3306'
DB = 'GLOBAL'
ID = 'root'
PW = 'root'

db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()

### 3. MFI 초기세팅
print(">> 3. Init MFI")

# 3.0. 기존 데이터 제거
print("3.0. Delete old data")
cursor.execute("TRUNCATE mfi")
db.commit()

# 2.1. 심볼리스트 조회
print("2.1. Get symbol list")
cursor.execute("SELECT DISTINCT symbol FROM stocks_price")
symbols = list(map(lambda x: x[0], cursor.fetchall()))

# 2.2. 기간 설정
print("2.2. Set period")
start = '20210801'
end = str(datetime.now().date()).replace('-', '')
print("Period: (%s - %s)" %(start, end))

# 3.3. 새로운 데이터 업데이트
print("2.3. Update new data (%s - %s)" %(start, end))
day = 10
for symbol in symbols:
  try:
    cursor.execute("SELECT * FROM stocks_price WHERE symbol = '" + symbol + "' ORDER BY date DESC")
    priceDf = pd.DataFrame(cursor.fetchall())
    priceDf = priceDf.rename(columns={0: 'date', 1:'symbol', 2:'open', 3:'high', 4:'low', 5:'close', 6:'volume'})
    # Set Dataframe
    priceDf['tp'] = 0.0
    priceDf['mfi'] = 0.0
    priceDf['mfi_diff'] = 0.0
    # Set typical price
    for idx in range(0, len(priceDf)):
      row = priceDf.iloc[idx]
      priceDf.at[idx, 'tp'] = (row['high'] + row['low'] + row['close']) / 3
    # Set MFI
    for idx in range(0, len(priceDf)):
      if idx + day > len(priceDf) - 1:
        continue
      positiveRMF = 0
      negativeRMF = 0          
      for i in range(idx, idx + day):
        today = priceDf.iloc[i]
        yesterday = priceDf.iloc[i + 1]
        if today['tp'] > yesterday['tp']:
          positiveRMF += today['tp'] * today['volume']
        elif today['tp'] < yesterday['tp']:
          negativeRMF += today['tp'] * today['volume']
      MFI = positiveRMF / (positiveRMF + negativeRMF) * 100
      priceDf.at[idx, 'mfi'] = MFI
      if idx > 0:
        priceDf.at[idx - 1, 'mfi_diff'] = priceDf.iloc[idx - 1]['mfi'] - MFI
    mfiDf = priceDf.reset_index().set_index('date').drop(['index', 'open', 'high', 'low', 'close', 'volume'], axis=1)
    db_connection = create_engine('mysql+pymysql://'+ ID +':'+ PW +'@'+ ADDR +':'+ PORT +'/'+ DB, encoding='utf-8', pool_pre_ping=True)
    conn = db_connection.connect()
    mfiDf.to_sql(name='mfi', con=db_connection, if_exists='append', index=True, index_label="date")        
    conn.close()
  except Exception as ex1:
    print('ex1', ex1, symbol)
    pass
db.close()

>> 3. Init MFI
3.0. Delete old data
2.1. Get symbol list
2.2. Set period
Period: (20210801 - 20211104)
2.3. Update new data (20210801 - 20211104)
ex1 unsupported operand type(s) for +: 'NoneType' and 'NoneType' BFB
ex1 unsupported operand type(s) for +: 'NoneType' and 'NoneType' BRKB


In [3]:
import pandas as pd
import pymysql
from datetime import datetime, timedelta
import traceback
import telegram

# DB 정보
ADDR = '192.168.56.100'
PORT = '3306'
DB = 'GLOBAL'
ID = 'root'
PW = 'root'

# 텔레그램 정보
bot_token = '2023613321:AAHCb61s8LZoCjRqSt9w-HOeGVh2jPOQ3go'
bot = telegram.Bot(token = bot_token)
chat_id = 2065271401
channel_chat_id = -1001722034898
## Boundary
position_bid = 0.1
position_ask = 0.8
mfi_bid_lv1 = 40
mfi_bid_lv2 = 30
mfi_bid_lv3 = 20
mfi_bid_lv4 = 10
mfi_ask = 90

### 1. 종목조회
print("\n>> 1. Get symbol")

# 1.1. 기간 설정
print("\n1.1. Set the date")
db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
cursor = db.cursor()
cursor.execute("SELECT date FROM stocks_price ORDER BY date DESC LIMIT 1")
date = str(cursor.fetchall()[0][0])
db.close()
# date = "20211020"
print("Date: (%s)" %(date))

try:
  # 1.2. 티커별 종합정보조회
  print("\n1.2. Get info tickers")
  sql = """
  SELECT BOLMFI.*
  FROM (
    SELECT BOL.*, MFI.tp, MFI.mfi, MFI.mfi_diff
    FROM (
      SELECT * 
      FROM boll 
      WHERE date = '%s'
    ) AS BOL
    JOIN (
      SELECT symbol, tp, mfi, mfi_diff
      FROM mfi
      WHERE date = '%s'
    ) AS MFI
    ON (BOL.symbol = MFI.symbol)
  ) AS BOLMFI
  """ % (date, date)
  
  db = pymysql.connect(host=ADDR, port=int(PORT), user=ID, passwd=PW, db=DB, charset='utf8')
  cursor = db.cursor()
  cursor.execute(sql)
  db.close()
  sumDf = pd.DataFrame(cursor.fetchall()).rename(columns={0:'date', 1:'symbol', 2:'close', 3:'low', 4:'medium', 5:'high', 6:'bandWidth', 7:'position', 8: 'tp', 9: 'mfi', 10: 'mfi_diff'})
  sumDf.set_index('symbol', drop=True, inplace=True)
  print(list(sumDf.index))
  lv1, lv2, lv3, lv4 = "", "", "", ""

  for idx in sumDf.index:        
    row = sumDf.loc[idx].copy()
    symbol = row.name
    ############################
    ## BID STEP    
    if row['position'] <= position_bid and row['mfi'] > 0 and row['mfi_diff'] > 0:
      if row['mfi'] <= mfi_bid_lv4:
        lv4 += "symbol: %s, boll: %.2f, MFI: %.2f\n" %(symbol, row['position'], row['mfi'])
      elif row['mfi'] <= mfi_bid_lv3:
        lv3 += "symbol: %s, boll: %.2f, MFI: %.2f\n" %(symbol, row['position'], row['mfi'])
      elif row['mfi'] <= mfi_bid_lv2:
        lv2 += "symbol: %s, boll: %.2f, MFI: %.2f\n" %(symbol, row['position'], row['mfi'])
      elif row['mfi'] <= mfi_bid_lv1:
        lv1 += "symbol: %s, boll: %.2f, MFI: %.2f\n" %(symbol, row['position'], row['mfi'])
    
    ############################
    ## ASK STEP    
    if symbol in ['NRGU, BTC/KRW']:      
      if row['mfi'] >= mfi_ask:
        # 2.1. 매도기회포착
        print("\n2.1. It's time to Ask!!! (%s)" %(symbol))
        bot.sendMessage(chat_id = chat_id, text="[%s] 매도기회포착 (%s)" %(symbol))
        bot.sendMessage(chat_id = chat_id, text="MFI: %.2f" %(row['mfi']))
        bot.sendMessage(chat_id = channel_chat_id, text="[%s] 매도기회포착 (%s)" %(symbol))
        bot.sendMessage(chat_id = channel_chat_id, text="MFI: %.2f" %(row['mfi']))

  # 3.1. 매수기회포착
  if len(lv1) > 0 or len(lv2) > 0 or len(lv3) > 0 or len(lv4) > 0:
    print("\n 3.1 It's time to bid !!!")
    bot.sendMessage(chat_id = chat_id, text="[%s] 매수기회포착" %(date))
    bot.sendMessage(chat_id = channel_chat_id, text="[%s] 매수기회포착" %(date))
  if len(lv1) > 0:
    print("Level 1")
    print(lv1)
    bot.sendMessage(chat_id = chat_id, text="[Level 1]")
    bot.sendMessage(chat_id = chat_id, text=lv1)
    bot.sendMessage(chat_id = channel_chat_id, text="[Level 1]")
    bot.sendMessage(chat_id = channel_chat_id, text=lv1)
  if len(lv2) > 0:
    print("Level 2")
    print(lv2)
    bot.sendMessage(chat_id = chat_id, text="[Level 2]")
    bot.sendMessage(chat_id = chat_id, text=lv2)
    bot.sendMessage(chat_id = channel_chat_id, text="[Level 2]")
    bot.sendMessage(chat_id = channel_chat_id, text=lv2)
  if len(lv3) > 0:
    print("Level 3")
    print(lv3)
    bot.sendMessage(chat_id = chat_id, text="[Level 3]")
    bot.sendMessage(chat_id = chat_id, text=lv3)
    bot.sendMessage(chat_id = channel_chat_id, text="[Level 3]")
    bot.sendMessage(chat_id = channel_chat_id, text=lv3)
  if len(lv4) > 0:
    print("Level 4")
    print(lv4)
    bot.sendMessage(chat_id = chat_id, text="[Level 4]")
    bot.sendMessage(chat_id = chat_id, text=lv4)
    bot.sendMessage(chat_id = channel_chat_id, text="[Level 4]")
    bot.sendMessage(chat_id = channel_chat_id, text=lv4)
except Exception as ex1:
  print('ex1', traceback.format_exc(), ex1)

# SELECT BOLMFI.*
# FROM (
#   SELECT BOL.*, MFI.tp, MFI.mfi, MFI.mfi_diff
#   FROM (
#     SELECT * 
#     FROM boll 
#     WHERE date = "20211020"
#   ) AS BOL
#   JOIN (
#     SELECT symbol, tp, mfi, mfi_diff
#     FROM mfi
#     WHERE date = "20211020"
#   ) AS MFI
#   ON (BOL.symbol = MFI.symbol)
# ) AS BOLMFI
# WHERE mfi <= 40
# AND mfi > 0
# AND position <= 0.1
# AND mfi_diff > 0
# ;


>> 1. Get symbol

1.1. Set the date
Date: (2021-11-05)

1.2. Get info tickers
['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABNB', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANET', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ASML', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZN', 'AZO', 'BA', 'BAC', 'BAX', 'BBWI', 'BBY', 'BDX', 'BIDU', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BNTX', 'BR', 'BRO', 'BSX', 'BWA', 'BXP', 'CAH', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CERN', 'CHRW', 'CHTR', 'CMCSA', 'CME', 'CNC', 'CNP', 'COF', 'COIN', 'COST', 'CPB', 'CRWD', 'CSCO', 'CSX', 'CTAS', 'CTLT', 'CTSH', 'CZR', 'DDOG', 'DOCU', 'DXCM', 'EA', 'EBAY', 'EQIX', 'EXC', 'FB', 'FISV', 'FNGU', 'FTNT', 'GILD', 'GOOG', 'GOOGL', 'HON', 'IDXX', 'ILMN', 'INTC', 'INTU', 'I

In [2]:
sumDf

""
